# Test the training process
This notebook is used to examine training process incase you are not familiar with the train

In [8]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Lambda
from tensorflow.python.keras.optimizers import SGD
import os,h5py,sys
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import random
import itertools
from nltk.metrics.distance import edit_distance
from scipy import misc

from PIL import Image
from tqdm import tqdm
import json
import os
import json
import numpy as np
import cv2
# from niq.util import memoize
import tensorflow as tf
from time import time
from text_normalizer.utils.normalize import normalize_text
from memoize import memoize
# from data_utils import fetch_path

In [3]:
def convert_text_label(input_string):
    input_string = input_string
    rt = []
    for ch in input_string:
        if ch in text_label.keys():
            lb = text_label[ch]
        else:
            raise Exception('{} does not exist, textline: {}'.format(ch, input_string))
        rt.append(lb)
    return rt

def fuse_dataset(dataset_1, dataset_2, dataset_1_prob=.8):
    """
        Args:
            dataset_1: instance of tf.Dataset
            dataset_2: instance of tf.Dataset
            dataset_1_prob: probability to return dataset1
        Returns:
            
    """
    tensors_1 = dataset_1.repeat(100).make_one_shot_iterator().get_next()
    tensors_2 = dataset_2.repeat(100).make_one_shot_iterator().get_next()
    prob_dataset_1 = tf.random_uniform(shape=[]) < dataset_1_prob
    features, tar, tar_str, typ = tuple(tf.cond(prob_dataset_1,
                                                lambda: (*tensors_1, 'dataset_1'),
                                                lambda:(*tensors_2, 'dataset_2'))
                                       )
    
    return features, tar, tar_str, typ

In [4]:
# Load new label_text dictionary
label_text = json.load(open('./logdir/box/trained_model/label_text.json', 'r'))
label_text = {int(k):v for k,v in label_text.items()}
text_label = {v:k for k, v in label_text.items()}

# Fetch synthetic paths

In [5]:
from data_utils import fetch_path_raw

In [6]:
data_dict_test = fetch_path_raw('./datasets/dc3/test/')
data_dict_train = fetch_path_raw('./datasets/dc3/train/')
data_dict_synthetic_test = fetch_path_raw('./datasets/systhetic_from_text_train/')

100%|██████████| 35992/35992 [00:00<00:00, 374864.70it/s]


./datasets/dc3/test/ 5548
./datasets/dc3/train/ 35992


100%|██████████| 100000/100000 [00:00<00:00, 360310.94it/s]


./datasets/systhetic_from_text_train/ 100000


In [ ]:
data_dict_kankush = fetch_path_raw('./datasets/kankuset/train')

In [ ]:
data_dict_train = data_dict_kankush + data_dict_synthetic_test*4

# tf Dataset

In [ ]:
def create_tf_dataset_location(ann_paths, batch_size = 2):
    """
        inputs: fetch path
        returns: tf dataset of, features, label_np, list_location, raw_path
    """
    from data_utils import read_image    
    def map_func(ann_path):

        with open(ann_path, 'r') as  f:
            lines = f.readlines()
            location_dict = eval(lines[0])

        path = ann_path[:-4]

        locations = []
        characters = []
        start_location = 0
        text = ''.join([_[-1] for _ in location_dict])
        strip_text = text.strip()
        s_strip = text.index(strip_text)
        e_strip = s_strip+len(strip_text) 

        for x1,x2, c in location_dict:
            start_location = x1//4
            end_location = x2//4
            locations.append([start_location, end_location])
            norm_char = normalize_text(c)
            norm_char_label = convert_text_label(norm_char)
            characters.append(norm_char_label)

        max_l = max([len(_) for _ in characters])
        characters_np = np.ones([len(characters), max_l], dtype=np.float32)*-1
        for i, character in enumerate(characters):
            characters_np[i,:len(character)] = character
        locations = np.array(locations, dtype=np.int32)
        return read_image(path), characters_np, locations , path




    dataset = tf.data.Dataset.from_tensor_slices(ann_paths)
    dataset = dataset.map(lambda item1: tf.py_func(
              map_func, [item1], [tf.float32, tf.float32, tf.int32, tf.string]), num_parallel_calls=8)

    dataset = dataset.padded_batch(batch_size, padded_shapes=([1, None, 512], [None, None], [None,2], []), 
                                        padding_values=(0., -1., -1, ''))

    dataset = dataset.prefetch(100)
    return dataset

In [ ]:
from data_utils import read_image
from pyson.vision import resize_by_factor

In [ ]:
def is_valid_string(string):
    for ch in string:
        if not ch in text_label.keys():
            return False
    return True

In [ ]:
def create_tf_dataset(tupple_path_annotations, batch_size = 16):
    
    def shuffle(paths, batch_size):
        rts = []
        for i in range(0,len(paths), batch_size):
            rts.append(paths[i:i+batch_size])
        np.random.shuffle(rts)
        rt = []
        for _ in rts:
            rt.extend(_)
        return rt
    
    def map_func(tupple_path_annotation):
        path, ann = eval(tupple_path_annotation)
        label_string = normalize_text(ann['text'])
        path = tf.compat.as_text(path,encoding='utf-8')
        label_number = convert_text_label(label_string)
        label_number_np = np.ones([len(label_number)]) * -1
        label_number_np[:] = label_number
        
        img = read_image(path)
        img = resize_by_factor(img, 48/img.shape[0])
        img = np.expand_dims(img, -1)
        img = img.astype(np.float32)
        return img, label_number_np.astype(np.float32)
    
    tupple_path_annotations = [_ for _ in tupple_path_annotations if is_valid_string(_)]
    
    tupple_path_annotations = shuffle(tupple_path_annotations, batch_size)
    
    tupple_path_annotations = [str(_) for _ in tupple_path_annotations]
    
    dataset = tf.data.Dataset.from_tensor_slices(tupple_path_annotations)
    dataset = dataset.map(lambda item1: tf.py_func(
              map_func, [item1], [tf.float32, tf.float32]), num_parallel_calls=8)

    dataset = dataset.padded_batch(batch_size, padded_shapes=([48, None, 1], [None]), 
                                        padding_values=(0., -1.))
    
    dataset = dataset.prefetch(100)
    return dataset


In [ ]:
batch_size = 32

In [ ]:
dataset_train = create_tf_dataset(data_dict_train, batch_size=batch_size)
dataset_test = create_tf_dataset(data_dict_test, batch_size=batch_size)

In [ ]:
img, label_number_np = dataset_test.make_one_shot_iterator().get_next()

In [ ]:
sess = tf.keras.backend.get_session()

In [ ]:
from pyson.vision import plot_images

In [ ]:
_img,_label_number_np = sess.run([img, label_number_np])

In [ ]:
_labels = []
for line in _label_number_np.astype(int):
    text = ''
    for ch in line:
        text += label_text[ch] if ch in label_text.keys() else ''
    _labels.append(text,)

In [ ]:
plot_images(_img[...,0], mxn=(16, 2))

# Model utils

In [ ]:
from utils.ctc_bestpath import ctcBestPath

def dump_model_config(output_path, model):
    config = model.get_config()
    with open(output_path, 'w') as f:
        json.dump(config, f)
    

def caculate_error_rate(features, tar_string):
    o = predict_with_session(features)
    t = [x.decode('utf-8') for x in tar_string]
    outs = np.array([edit_distance(a,b) for a, b in zip(t, o)])
    outs = outs.clip(0,1)
    return 1-outs.mean()


# Create inputs tensors

In [ ]:
def predict_model(kush_model, decoder):
    input = kush_model.input
    x = input
    for layer in kush_model.layers[:-1]:
        x = layer(x)
    return tf.keras.models.Model(input, x)



In [ ]:
# K.clear_session()
# tf.reset_default_graph()
# sess = K.get_session()

In [32]:
weights_path = './logdir/box/trained_model/basemodel_retrain_50000.h5'
config_path = './logdir/box/trained_model/basemodel.json'


def basemodel_with_batch_norm(basemodel_old):
    
    '''
    
    timedistrib (TimeDistributed (None, None, 512)         0         
    _________________________________________________________________
    blstm1 (Bidirectional)       (None, None, 512)         1182720   
    _________________________________________________________________
    blstm1_out (Dense)           (None, None, 256)         131328    
    _________________________________________________________________
    blstm2 (Bidirectional)       (None, None, 512)         789504    
    _________________________________________________________________
    dense (Dense)                (None, None, 4166)        2137158   
    ================================================================
'''
    train_vars_list = []
    input = basemodel_old.input
    x = input
    for layer in basemodel_old.layers[1:]:
        if layer.name == 'timedistrib':
            break
        x = layer(x)
        
    x = basemodel_old.get_layer('timedistrib').output
    x = tf.keras.layers.BatchNormalization()(x)
    x = basemodel_old.get_layer('blstm1')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = basemodel_old.get_layer('blstm1_out')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = basemodel_old.get_layer('blstm2')(x)
    x = basemodel_old.get_layer('dense')(x)
    basemodel = tf.keras.models.Model(input, x)
    is_train_layer = False
    for layer in basemodel.layers:
        if layer.name == 'timedistrib':
#             import ipdb; ipdb.set_trace()
            is_train_layer = True
        if is_train_layer:
            train_vars_list.extend(layer.weights)
            
    return basemodel, train_vars_list

config = json.load(open(config_path, 'r'))
basemodel_4166 = tf.keras.models.Model.from_config(config)
basemodel_4166.load_weights(weights_path)

basemodel, train_vars_list = basemodel_with_batch_norm(basemodel_4166)

basemodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, None, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 48, None, 64)      640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 24, None, 64)      0         
_________________________________________________________________
drop1 (Dropout)              (None, 24, None, 64)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 24, None, 128)     73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 12, None, 128)     0         
_________________________________________________________________
drop2 (Dropout)              (None, 12, None, 128)     0         
__________

In [33]:
train_vars_list

[<tf.Variable 'batch_normalization_29/gamma:0' shape=(512,) dtype=float32>,
 <tf.Variable 'batch_normalization_29/beta:0' shape=(512,) dtype=float32>,
 <tf.Variable 'batch_normalization_29/moving_mean:0' shape=(512,) dtype=float32>,
 <tf.Variable 'batch_normalization_29/moving_variance:0' shape=(512,) dtype=float32>,
 <tf.Variable 'blstm1_24/forward_cu_dnngru_6/kernel:0' shape=(512, 768) dtype=float32>,
 <tf.Variable 'blstm1_24/forward_cu_dnngru_6/recurrent_kernel:0' shape=(256, 768) dtype=float32>,
 <tf.Variable 'blstm1_24/forward_cu_dnngru_6/bias:0' shape=(1536,) dtype=float32>,
 <tf.Variable 'blstm1_24/backward_cu_dnngru_6/kernel:0' shape=(512, 768) dtype=float32>,
 <tf.Variable 'blstm1_24/backward_cu_dnngru_6/recurrent_kernel:0' shape=(256, 768) dtype=float32>,
 <tf.Variable 'blstm1_24/backward_cu_dnngru_6/bias:0' shape=(1536,) dtype=float32>,
 <tf.Variable 'batch_normalization_30/gamma:0' shape=(512,) dtype=float32>,
 <tf.Variable 'batch_normalization_30/beta:0' shape=(512,) dtype

In [ ]:
train_images, train_labels = dataset_train.repeat(100).make_one_shot_iterator().get_next()

In [ ]:
test_images, test_labels = dataset_test.repeat(100).make_one_shot_iterator().get_next()

In [ ]:
from pyson.losses import suppervise_ctc_loss, ctc_loss

In [ ]:
train_preds = basemodel_4166(train_images)

normal_ctc_loss = ctc_loss(train_labels, train_preds)
loss = normal_ctc_loss


global_step = tf.Variable(0, dtype=tf.int64, name='global_step')
train_step = tf.train.AdamOptimizer().minimize(loss, var_list=basemodel_4166.layers[-1].weights, global_step=global_step)

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
basemodel_4166.load_weights(weights_path)

# Summary

In [ ]:
losses = {'supervise_loss':[], 'normal_ctc_loss': [], 'train_acc':[], 'test_acc':[]}

In [ ]:

def pretty_summary():
    s = ''
    for k, v in losses.items():
        s += '{}:{:0.4f}\t'.format(k, np.mean(v[-100:]))
    return s

def update_summary(supervise_loss, normal_ctc_loss, train_acc, test_acc):
    losses['supervise_loss'].append(supervise_loss)
    losses['normal_ctc_loss'].append(normal_ctc_loss)
    losses['train_acc'].append(train_acc)
    losses['test_acc'].append(test_acc)

In [ ]:
test_preds = basemodel_4166(test_images)
def caculate_error_rate(mode):
    global test_preds
    global train_preds
    def get_texts(tar):
        texts = []
        for line in tar:
            text = ''
            line = np.reshape(line, [-1]).astype(int)
            for ichar in line:
                if ichar != -1:
                    text += label_text[ichar]
            texts.append(text)
        return texts
    
    if mode == 'test':
        o, t = sess.run([test_preds, test_labels])
    elif mode == 'train':
        o, t = sess.run([train_preds, train_labels])
    else:
        raise Exception('not yet implemented')
    t = get_texts(t)#[_.decode('utf-8') for _ in t]
    o = [ctcBestPath(_, label_text) for _ in o]
    outs = np.array([edit_distance(a,b) for a, b in zip(t, o)])
    outs = outs.clip(0,1)
    return 1-outs.mean(), t, o

In [ ]:
LOGDIR = 'logdir/28_feb/'

In [ ]:
# tf.summary.scalar('supervise_loss', supervise_loss)
tf.summary.scalar('normal_ctc_loss', normal_ctc_loss)
merged = tf.summary.merge_all()
tensor_dir = LOGDIR + '/tb'
os.system('rm -r {}/*'.format(tensor_dir))
train_writer = tf.summary.FileWriter(tensor_dir,sess.graph)
print('RUN: tensorboard --logdir {} --port 5001'.format(os.path.abspath(LOGDIR)))
summary = tf.Summary()


In [ ]:
def get_char_set(data_dict):
    texts = [_[1]['text'] for _ in data_dict]
    chars = ''.join(texts)
    chars_norm = normalize_text(chars)
    return set(chars_norm)
train_chars = get_char_set(data_dict_train)
test_chars = get_char_set(data_dict_test)

In [ ]:
train_chars_kan = get_char_set(data_dict_kankush)
print(len(train_chars), len(test_chars))
unavail = test_chars - train_chars - train_chars_kan

In [ ]:
# for text in  [_[1]['text'] for _ in data_dict_test]:
#     for ch in unavail:
#         if ch in text:
#             print(text, '\t', ch)

In [ ]:
display_freq = 5
start = time()
init_step = sess.run(global_step)
for step in range(init_step, int(32e5)):
    train_dict = {'train_step':train_step}
    if step % display_freq == 0:
        train_dict['norm_loss'] = normal_ctc_loss
    
    results = sess.run(train_dict)
    if step % display_freq == 0:
        speed = (step-init_step) * batch_size / (time()-start)
        _summary = sess.run(merged)
        train_writer.add_summary(_summary, step)
        train_accuracy = caculate_error_rate('train')[0]
        test_accuracy = caculate_error_rate('train')[0]
        summary.value.add(tag='Train Accuracy', simple_value=train_accuracy)
        summary.value.add(tag='Test Accuracy', simple_value=test_accuracy)

#         _supervise_loss, _normal_loss = K.get_session().run([supervise_loss, normal_ctc_loss])        
        update_summary(0, results['norm_loss'], train_accuracy, test_accuracy)
        pretty_summary()
        
        print('\r Step: {}\t\t Speed: {:0.2f}\t {}'.format(step, speed, pretty_summary()), end='')
        train_writer.add_summary(summary, step)

#     if step % 5000 == 0 and step > 100:
#         save_path = os.path.join(LOGDIR, 'basemodel_retrain_{}.h5'.format(step))
#         print('save at: ', save_path)
#         basemodel.save_weights(save_path)
#         saver.save(sess, LOGDIR, global_step=global_step)

In [ ]:
save_path = os.path.join(LOGDIR, 'basemodel_retrain_{}.h5'.format(step))
print('save at: ', save_path)
basemodel.save_weights(save_path)
# saver.save(sess, LOGDIR, global_step=global_step)

# Prediction

In [ ]:
# from glob import glob
# paths = glob('./datasets/kankuset/train/raw/*.kush.npy')
# print(len(paths))
# from tqdm import tqdm
# import os
# !mkdir ./datasets/kankuset/train/kush
# for path in tqdm(paths):
#     new_path = paths[0].replace('.kush', '').replace('/raw/', '/kush/')#paths[0].strip('.kush').replace('/raw/','/kush/')
#     os.rename(path, new_path)